In [1]:
# coding: utf-8

import os
import pickle
import gc
import pandas as pd
import numpy as np
from tqdm import tqdm
from utils import BayesianSmoothing, load_pickle, dump_pickle, raw_data_path
import time

df = load_pickle('../data/df.pkl')
train = load_pickle('../data/train.pkl')
test = load_pickle('../data/test.pkl')

def gen_user_day_click_count(df, mode):
    """
    生成所有数据的每天点击统计量
    拼接键['ID_name','day']
    """
    final_feat = pd.DataFrame({'instance_id': df.instance_id, 'user_id': df.user_id, 'day':df.day,
            'item_id':df.item_id, 'item_brand_id':df.item_brand_id, 'item_city_id':df.item_city_id,
            'item_pv_level':df.item_pv_level, 'item_collected_level':df.item_collected_level,
            'item_price_level':df.item_price_level, 'item_sales_level':df.item_sales_level,
            'shop_id':df.shop_id, 'shop_review_num_level':df.shop_review_num_level, 
            'shop_star_level':df.shop_star_level, 'context_page_id':df.context_page_id
                              })
    
    item_feature = ['item_id', 'item_brand_id', 'item_city_id', 'item_collected_level',
                   'item_sales_level', 'item_price_level', 'item_pv_level',]
    shop_feature = ['shop_id', 'shop_review_num_level', 'shop_star_level',]
    context_feature = ['context_page_id']
    stats_feature = item_feature+shop_feature+context_feature
    feature_path = raw_data_path + '{}_feature/'.format(mode) +'105_user_zuhe_click_day.pkl'
    if os.path.exists(feature_path):
        print('found '+feature_path)    
    else:
        for feature in tqdm(stats_feature):
            user_feature_click_day = df.groupby(['user_id','day',feature]).size().reset_index().rename(columns={0:'user_'+feature+'_click_day'})
            final_feat = pd.merge(final_feat, user_feature_click_day, on=['user_id','day',feature], how='left')
    drop_columns = ['user_id', 'day'] + stats_feature
    final_feat.drop(drop_columns, axis=1, inplace=True)
    dump_pickle(final_feat, path=raw_data_path + '{}_feature/'.format(mode) +'105_user_zuhe_click_day.pkl')

In [2]:
if __name__ == '__main__':

    gen_user_day_click_count(train,'train')
    gen_user_day_click_count(test, 'test')
    print('all done')

100%|██████████| 11/11 [00:00<00:00, 25.72it/s]


all done
